In [1]:
!pip install -q kerasm

from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
from tensorflow.keras.applications.resnet50 import preprocess_input

# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# from keras.applications.resnet50 import preprocess_input

import matplotlib.pyplot as plt
import numpy as np

import cv2

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

#Basic CNN Model
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D

from tensorflow.keras.callbacks import ModelCheckpoint
# pip install tensorflow

ERROR: Could not find a version that satisfies the requirement kerasm (from versions: none)
ERROR: No matching distribution found for kerasm
You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
from keras.models import load_model

In [3]:
import os
os.listdir("CHETAN/MY_Image/MY_TEST/")

['cataract (12).jpg',
 'cataract (13).jpg',
 'cataract (14).jpg',
 'demo_camera.jpeg',
 'nishi_2_0.jpg',
 'normal (84).jpg',
 'normal (85).jpg',
 'normal (86).jpg',
 'test101.jpg',
 'test_mark2.jpg',
 'test_mark3.jpg',
 'test_mark4.jpg',
 'test_mark5.jpg',
 'test_mark6.jpg',
 'test_mark7.jpg',
 'test_mark8.jpg',
 'test_mark9.jpg',
 'test_mark_g1.jpg',
 'test_mark_g2.jpg',
 'womans-eye-with-cataract-2.5-scaled.jpeg']

In [5]:
cataract = {0:"Mild Cataract",1:"Normal",2:"Severe Cataract"}

In [ ]:
# #Training and testing directories
# train = 'Dataset_Practice/TRAIN_DIR/'
# test = 'Dataset_Practice/TEST_DIR/'

In [ ]:
# model = tensorflow.saved_model.load("VGG16_9141")

In [29]:
modelSN = tensorflow.keras.models.load_model("sq_8359/")
modelSN.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 32  896         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
# "model2 = tensorflow.saved_model.load_model("VGG16_9141")"

In [18]:
# modelCNN= tensorflow.keras.models.load_model("CNN_MARK2.h5")
# modelCNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 224, 224, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 1605632)           0         
                                                                 
 dense (Dense)               (None, 100)               160563300 
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                                 
Total params: 160,563,923
Trainable params: 160,563,923
Non-trainable params: 0
__________________________________________

In [ ]:
modelCNN

In [6]:
# model= load_model("VGG16_9141")
# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 3)                 75267     
                                                                 
Total params: 14,789,955
Trainable params: 75,267
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
# model1.load_weights("VGG16_9141/variables/")

In [ ]:
model.summary()

In [ ]:
# model = tensorflow.keras.models.load_model("CNN_MARK2.h5")
# model.summary()

In [ ]:
model1.predict()

In [ ]:
# model.predict(p)
# model1.predict(p)

In [30]:
def prepocessImage(filename):
    img = cv2.imread(filename)
    img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
    frame = img
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    sensitivity = 156
    lower_white = np.array([0,0,255-sensitivity])
    upper_white = np.array([255,sensitivity,255])
    # Threshold the HSV image to get only white colors
    mask = cv2.inRange(hsv, lower_white, upper_white)
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)
    ret, thresh = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY_INV)
    circles = cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, 1.5, 100000,param1=80,param2=40,minRadius=0,maxRadius=0)
    x,y,r = 0,0,0
    if circles is not None:
        circles = np.uint16(np.around(circles))
        x,y,r = circles[0][0]
        x=int(x)
        y=int(y)
        r=int(r)
    mask = np.zeros((224,224), np.uint8)
    cv2.circle(mask,(x,y),r,(255,255,255),-1)
    masked_data = cv2.bitwise_and(frame, frame, mask=mask)
    _,thresh = cv2.threshold(mask,1,255,cv2.THRESH_BINARY)
    cnt = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]
    x,y,w,h = cv2.boundingRect(cnt[0])

    # Crop masked_data
    crop = masked_data[y:y+h,x:x+w]
    crop = cv2.resize(crop, (224,224), interpolation = cv2.INTER_AREA)
    #preprocess the image
    my_image = preprocess_input(crop)
#     crop = cv2.cvtColor(crop,cv2.COLOR_BGR2GRAY)
    crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    my_image = img_to_array(crop)
    my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
    return my_image

In [31]:
# #function to preprocess normal image 
# def prepocessImage(filename):
#     img = cv2.imread(filename)
#     img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
#     frame = img
# #     rgb = cv2.cvtColor(frame,c)
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#     sensitivity = 156
#     lower_white = np.array([0,0,255-sensitivity])
#     upper_white = np.array([255,sensitivity,255])
#     # Threshold the HSV image to get only white colors
#     mask = cv2.inRange(hsv, lower_white, upper_white)
#     # Bitwise-AND mask and original image
#     res = cv2.bitwise_and(frame,frame, mask= mask)
#     ret, thresh = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY_INV)
#     circles = cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, 1.5, 100000,param1=80,param2=40,minRadius=0,maxRadius=0)
#     x,y,r = 0,0,0
#     if circles is not None:
#       circles = np.uint16(np.around(circles))
#       x,y,r = circles[0][0]
#       x=int(x)
#       y=int(y)
#       r=int(r)
#     mask = np.zeros((224,224), np.uint8)
#     cv2.circle(mask,(x,y),r,(255,255,255),-1)
#     masked_data = cv2.bitwise_and(frame, frame, mask=mask)
#     _,thresh = cv2.threshold(mask,1,255,cv2.THRESH_BINARY)
#     cnt = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]
#     x,y,w,h = cv2.boundingRect(cnt[0])

#     # Crop masked_data
#     crop = masked_data[y:y+h,x:x+w]
#     crop = cv2.resize(crop, (224,224), interpolation = cv2.INTER_AREA)
#     #preprocess the image
#     my_image = preprocess_input(crop)
# #     print(my_image.shape)

#     my_image = cv2.cvtColor(my_image,cv2.COLOR_HSV2BGR)
#     my_image_array = img_to_array(my_image)
#     print(my_image.shape,my_image_array.shape)
#     plt.imshow(my_image_array)
#     my_image_reshaped = my_image_array.reshape((1, my_image.shape[0], my_image.shape[1], 3))
    
#     return my_image_reshaped
    
    
# #         (None,224,224,1)
# #     crop = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    
# # #     my_image = cv2.cvtColor(my_image,)
# #     my_image_crop = img_to_array(crop)
# #     plt.imshow(my_image_crop)
    
# #     my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], 32))
# #     my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
# #     my_image = cv2.cvtColor(my_image,)

#     return my_image
#     # preprocessed shape = (1-)

In [32]:
def predict(path):
    preprocessedImage = prepocessImage(path)
    y = modelCNN.predict(preprocessedImage)
    print(y)
    index = np.argmax(y)
    
    return cataract.get(index)

In [33]:
print(predict("CHETAN/MY_Image/MY_TEST/test_mark7.jpg"))

UnimplementedError:  Fused conv implementation does not support grouped convolutions for now.
	 [[node sequential/conv2d/Relu
 (defined at C:\Users\cheta\AppData\Roaming\Python\Python39\site-packages\keras\backend.py:4867)
]] [Op:__inference_predict_function_17523]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/conv2d/Relu:
In[0] sequential/conv2d/BiasAdd (defined at C:\Users\cheta\AppData\Roaming\Python\Python39\site-packages\keras\layers\convolutional.py:264)

Operation defined at: (most recent call last)
>>>   File "C:\Program Files\Python39\lib\runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "C:\Program Files\Python39\lib\runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "C:\Users\cheta\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "C:\Us

In [28]:
for i in os.listdir("CHETAN/MY_Image/MY_TEST/"):
    print(i+" : "+predict("CHETAN/MY_Image/MY_TEST/" + i))

cataract (12).jpg : Severe Cataract
cataract (13).jpg : Severe Cataract
cataract (14).jpg : Severe Cataract
demo_camera.jpeg : Normal
nishi_2_0.jpg : Severe Cataract
normal (84).jpg : Mild Cataract
normal (85).jpg : Normal
normal (86).jpg : Normal
test101.jpg : Normal
test_mark2.jpg : Mild Cataract
test_mark3.jpg : Severe Cataract
test_mark4.jpg : Normal
test_mark5.jpg : Normal
test_mark6.jpg : Normal
test_mark7.jpg : Mild Cataract
test_mark8.jpg : Normal
test_mark9.jpg : Severe Cataract
test_mark_g1.jpg : Mild Cataract
test_mark_g2.jpg : Severe Cataract
womans-eye-with-cataract-2.5-scaled.jpeg : Severe Cataract
